In [1]:
#Import liberaries
import pandas as pd
import numpy as np

In [2]:
from scipy.stats import skew, kurtosis

In [3]:
from scipy.stats import randint

In [4]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix

In [8]:
from sklearn.model_selection import train_test_split

In [21]:
from sklearn.model_selection import StratifiedKFold, cross_val_score,cross_validate
from sklearn.ensemble import RandomForestClassifier

from sklearn.pipeline import make_pipeline

In [5]:
#load data 
df_other = pd.read_csv("behavior_data.csv")
# Prepare features and labels
X_other = df_other.drop(columns='Target')
y_other = df_other['Target']

In [6]:
print(df_other['Target'].value_counts())

Target
3    27
1    24
2    23
4    18
Name: count, dtype: int64


In [10]:
# Train-test split (use stratify to maintain class balance)
X_train_otherM, X_test_otherM, y_train_otherM, y_test_otherM = train_test_split(
    X_other, y_other, test_size=0.2, random_state=42, stratify=y_other
)

In [ ]:
# Standardize features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_otherM)
X_test_scaled = scaler.transform(X_test)

In [11]:
# Train logistic regression
logregB = LogisticRegression(max_iter=1000, multi_class='multinomial', solver='lbfgs')
logregB.fit(X_train_otherM, y_train_otherM)

LogisticRegression(max_iter=1000, multi_class='multinomial')

In [12]:
# Predict and evaluate
y_pred_LogR = logregB.predict(X_test_otherM)
print("Classification Report:\n")
print(classification_report(y_test_otherM, y_pred_LogR))

Classification Report:

              precision    recall  f1-score   support

           1       1.00      0.80      0.89         5
           2       1.00      0.80      0.89         5
           3       0.67      0.80      0.73         5
           4       0.60      0.75      0.67         4

    accuracy                           0.79        19
   macro avg       0.82      0.79      0.79        19
weighted avg       0.83      0.79      0.80        19



In [13]:
#####Let's try other classification modules 
from xgboost import XGBClassifier
#Trying XGB classifier
# Adjust labels to be 0-based
y_train_adjt = y_train_otherM - 1
y_test_adjt = y_test_otherM - 1
XgbclfB = XGBClassifier(n_estimators=100, max_depth=5, learning_rate=0.1, use_label_encoder=False, eval_metric='mlogloss')
XgbclfB.fit(X_train_otherM, y_train_adjt)

C:\Users\labuser\anaconda3\Lib\site-packages\xgboost\training.py:183: UserWarning: [21:05:34] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='mlogloss',
              feature_types=None, feature_weights=None, gamma=None,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=5, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, ...)

In [14]:
# Predict and evaluate
y_pred_XGB = XgbclfB.predict(X_test_otherM)
print(classification_report(y_test_adjt, y_pred_XGB))

              precision    recall  f1-score   support

           0       0.71      1.00      0.83         5
           1       1.00      0.60      0.75         5
           2       0.67      0.80      0.73         5
           3       1.00      0.75      0.86         4

    accuracy                           0.79        19
   macro avg       0.85      0.79      0.79        19
weighted avg       0.84      0.79      0.79        19



In [28]:
# Features and target
X_p = df_other.drop(columns='Target')
y_p = df_other['Target']

# Create a pipeline (scaling + classifier)
pipeline = make_pipeline(
    StandardScaler(), 
    RandomForestClassifier(n_estimators=156, max_depth=11, random_state=42)
)

# 5-fold Stratified CV
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Evaluate model with cross-validation
scores = cross_val_score(pipeline, X_p, y_p, cv=cv, scoring='accuracy')

# Print results
print("Cross-validation accuracy scores:", scores)
print("Average accuracy:", scores.mean())

Cross-validation accuracy scores: [0.84210526 0.94736842 0.88888889 0.83333333 0.72222222]
Average accuracy: 0.8467836257309941


In [18]:
from xgboost import XGBClassifier
XGB_modal = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss', random_state=42)

In [25]:
from sklearn.svm import SVC

svcclf = SVC(kernel='rbf', C=1, gamma='scale')


In [24]:
# Features and target
# Adjust labels to be 0-based

y_adj = y_p - 1
# Replace model here with any of the ones above
pipeline = make_pipeline(
    StandardScaler(),
    XGB_modal  # insert chosen model
)

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

results = cross_validate(pipeline, X_p, y_adj, cv=cv, scoring=['accuracy', 'f1_macro'])

print("Avg Accuracy:", results['test_accuracy'].mean())
print("Avg F1 Score:", results['test_f1_macro'].mean())

C:\Users\labuser\anaconda3\Lib\site-packages\xgboost\training.py:183: UserWarning: [21:24:42] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
C:\Users\labuser\anaconda3\Lib\site-packages\xgboost\training.py:183: UserWarning: [21:24:42] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
C:\Users\labuser\anaconda3\Lib\site-packages\xgboost\training.py:183: UserWarning: [21:24:42] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
C:\Users\labuser\anaconda3\Lib\site-packages\xgboost\training.py:183: UserWarning: [21:24:43] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.up

Avg Accuracy: 0.8251461988304094
Avg F1 Score: 0.820108225108225


C:\Users\labuser\anaconda3\Lib\site-packages\xgboost\training.py:183: UserWarning: [21:24:43] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


In [26]:
# Features and target
# Adjust labels to be 0-based

# Replace model here with any of the ones above
pipeline = make_pipeline(
    StandardScaler(),
    svcclf# insert  model
)

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

results = cross_validate(pipeline, X_p, y_adj, cv=cv, scoring=['accuracy', 'f1_macro'])

print("Avg Accuracy:", results['test_accuracy'].mean())
print("Avg F1 Score:", results['test_f1_macro'].mean())

Avg Accuracy: 0.8362573099415205
Avg F1 Score: 0.8292207792207792
